In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score


In [56]:
x, y = load_breast_cancer(return_X_y=True,as_frame=False)
x

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [57]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [58]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, train_size=0.70, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.50, random_state=42)

In [ ]:
class TreeNode:
    def __init__(self, left=None, right=None, feature=None, feature_value=None, threshold=None):
        self.left = left
        self.right = right
        self.feature = feature
        self.feature_value = feature_value
        self.threshold = threshold

    def is_leaf(self): 
        return self.feature_value is not None # node becomes leaf when it has a value

In [ ]:
class DecisionTree:
    def __init__(self, max_depth, min_samples_split):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def build_tree(self, x, y, depth):
        samples, _ = x.shape # x dimensions is the samples with their features
        classes = len(np.unique(y))
        if len(np.unique(y)) == 1: # 1 class -> leaf node
            selected_class = y[0]
            return TreeNode(None, None, None, selected_class, None)
        
        if(samples < self.min_samples_split or depth >= self.max_depth or classes == 1): # stopping conditions
            selected_class = np.bincount(y).argmax()
            return TreeNode(None, None, None, selected_class, None)
        
        best_feature, best_threshold = self.select_best_split(x, y) 
        left_index, right_index = self.split_node(x, best_feature, best_threshold) # splitting the node at the best feature
        left_subtree = self.build_tree(x[left_index, :], y[left_index], depth + 1)
        right_subtree = self.build_tree(x[right_index, :], y[right_index], depth + 1)
        return TreeNode(left_subtree, right_subtree, best_feature, None, best_threshold) # return new node
        
    def calculate_Entropy(self, y):
        counts = np.bincount(y) # counts of each class in y
        probabilities = counts / counts.sum()
        entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
        return entropy
        
    def split_node(self, x, feature, threshold):
        left_index = x[:, feature] <= threshold # splitting samples at that feature and threshold
        right_index = x[:, feature] > threshold

        return left_index, right_index  

    def calculate_Information_Gain(self, x, y, feature, threshold):
        original_entropy = self.calculate_Entropy(y) #H(y)
        left_index, right_index = self.split_node(x, feature, threshold)
        n, n_left, n_right = len(y), len(y[left_index]), len(y[right_index])
        if n_left == 0 or n_right == 0: # no split
            return 0
        new_entropy = (n_left / n) * self.calculate_Entropy(y[left_index]) + (n_right / n) * self.calculate_Entropy(y[right_index]) # entropies * their weights
        information_gain = original_entropy - new_entropy
        return information_gain

    def select_best_split(self, x, y):
        best_information_gain = -1
        best_feature, best_threshold = None, None
        features = x.shape[1]

        for feature in range(features):
            thresholds = np.unique(x[:, feature])
            for threshold in thresholds:
                information_gain = self.calculate_Information_Gain(x, y, feature, threshold)
                if information_gain > best_information_gain:
                    best_information_gain = information_gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold
    
    def fit(self, x, y):
        self.root = self.build_tree(x, y, 0) # building the tree starting from depth 0
    
    def traverse_tree(self, x, node): # for prediction
        if node.is_leaf():
            return node.feature_value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        else:
            return self.traverse_tree(x, node.right)
        
    def predict(self, x):
        predictions = []
        for sample in x:
            prediction = self.traverse_tree(sample, self.root)
            predictions.append(prediction)
        return np.array(predictions)
        


In [61]:
dTree = DecisionTree(max_depth=5, min_samples_split=10)
dTree.fit(x_train, y_train)
ypred = dTree.predict(x_val)
accuracy = accuracy_score(y_val, ypred)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


Validation Accuracy: 92.94%
